This code includes a new route /download which allows the user to download the text file generated by the application. It reads the contents of the file videoText.txt and returns it as a response with the appropriate headers.

The if __name__ == '__main__': block at the end of the script ensures that the Flask application is started when the script is run as the main program. The debug=True argument enables debug mode, which can be useful for debugging the application during development.

# first try

In [ ]:
from flask import Flask, request, render_template, jsonify
import speech_recognition as sr
import os
import moviepy.editor as mp
from summa import summarizer
from spellchecker import SpellChecker
from python_translator import Translator

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/convert', methods=['POST'])
def convert():
    try:
        file = request.files['video-file']
        if request.form['convert'] == 'Convert':
            # path by wav
            path = os.path.splitext(file.filename)[0] + '.wav'

            # use Speech Recognition library to convert the audio of the video to text
            clip = mp.VideoFileClip(file)
            clip.audio.write_audiofile(path)
            r = sr.Recognizer()
            audio = sr.AudioFile(path)
            with audio as source:
                audio_file = r.record(source)
            # return the text
            try:
                lang = request.form['lang'] # added variable for language selection
                text = r.recognize_google(audio, language='ar-AR' if lang == 'arabic' else 'en-US')
                print('the video contains: ' + text)
                f4 = open('videoText.txt', 'a', encoding='utf-8')
                f4.writelines("The video text: \n" + text + "\n\n") 
                f4.close()
            except sr.UnknownValueError as u:
                print(u)
            except sr.RequestError as r:
                print(r)

            service = request.form['services'] # added variable for service selection
            if service == 'summarize':
                summarized_text = summarizer.summarize(text, language='ar' if lang == 'arabic' else 'en')
                print("Summarized text is: " + summarized_text)
                # save the summarized video text in txt file
                f4 = open('videoText.txt', 'a', encoding='utf-8')
                f4.writelines("The summarized text: \n" + summarized_text + "\n\n") 
                f4.close()

            elif service == 'correct':   
                corrector = SpellChecker(language='ar' if lang == 'arabic' else 'en')
                words = text.split()
                correct_words = []
                for word in words:
                    if word not in corrector:
                        correct_words.append(corrector.correction(word))
                corrected_text = " ".join(correct_words)
                print("Corrected text is: " + corrected_text)
                # save the corrected video text in txt file
                f4 = open('videoText.txt', 'a', encoding='utf-8')
                f4.writelines("The corrected text: \n" + corrected_text + "\n\n") 
                f4.close()

            elif service == 'translate':
                translator = Translator()
                dest_lang = 'ar' if lang == 'english' else 'en'
                result = translator.translate(text, dest=dest_lang)
                print("Translated text is: " + str(result))
                # save the translated video text in txt file
                f4 = open('videoText.txt', 'a', encoding='utf-8')
                f4.writelines("The translated text: \n" + str(result) + "\n\n") 
                f4.close()

        return jsonify({'success': True}) # added return statement
    except Exception as e: # added exception handling
        return jsonify({'success': False, 'error': str(e)})
    

@app.route('/download', methods=['GET'])
def download():
    try:
        with open('videoText.txt', 'r', encoding='utf-8') as f:
            contents = f.read()
        return Response(
            contents,
            mimetype="text/plain",
            headers={"Content-Disposition":
                     "attachment;filename=videoText.txt"})
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})


if __name__ == '__main__':
    app.run(debug=True)



# second try

In [ ]:
from flask import Flask, request, render_template
import speech_recognition as sr
import os
import  moviepy.editor as mp
import summa
from summa import summarizer
from spellchecker import SpellChecker
from python_translator import Translator

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_video():
    # check if file was uploaded
    if 'video-file' not in request.files:
        return render_template('index.html', error='No file uploaded')

    file = request.files['video-file']
# check if the file is a video
    if file.content_type.split('/')[0] != 'video':
        return {'error': 'Invalid file format. Please upload a video file.'}
 
 #check language of video from user 
    lang_of_video= request.form['lang-select']
    if lang_of_video=="en":

        # transcribe text from video
        text = convert_video_to_text(file)

        # summarize text
        summary = summarize_text(text)

        # correct spelling in text
        corrected_text = correct_spelling(text)

        # translate text to arabic
        translated_text = translate_text(text)

        # save text to file
        save_text_to_file(text)

        # render results page
        return render_template('index.html', text=text-output, summary=service1, corrected_text=service2, translated_text=service3,save_text_to_file=service4)



        def convert_video_to_text(file):
            #path by wav
            path= os.path.splitext(file)[0]+'.wav'

            # use Speech Recognition library to convert the audio of the video to text
            clip=mp.VideoFileClip(file)
            clip.audio.write_audiofile(path)
            r=sr.Recognizer()
            audio=sr.AudioFile(path)
            with audio as source:
                audio_file=r.record(source)
            # return the text
            try:
                text=r.recognize_google(audio_file,language='eng')
                return {'the video contains :': r.recognize_google(text)}
            except sr.UnknownValueError as u:
                print(u)
            except sr.RequestError as r:
                print(r)

        def summarize_text(text):
            #summarize text
            #sometimes if the text not big it will make empty text
            #text must not be too small
            summaized_text=summarizer.summarize(text, language ="english")
            return {"summarized text is: ": summaized_text}
            #save the summarized video text in txt file
            f4=open('videoText_EN.txt','a',encoding='utf-8')
            f4.writelines("The summarized text : \n"+summaized_text+"\n\n") 
            f4.close()
            print ("Done")


        def correct_spelling(text):
            corrector = SpellChecker()
            word = text
            if word in corrector:
                return "Correct"
            else:
                correct_word = corrector.correction(word)
                return {"Correct Spelling is: ": correct_word}
            #save the corrected video text in txt file
            f4=open('videoText_EN.txt','a',encoding='utf-8')
            f4.writelines("The corrected text : \n"+correct_word+"\n\n") 
            f4.close()
            print ("Done")


        def translate_text(text):
            translator = Translator()
            result = translator.translate(text, "arabic", "english")
            return {"Teranslated text : ": result}
            #save the translated video text in txt file
            f4=open('videoText_EN.txt','a',encoding='utf-8')
            f4.writelines("The translated text : \n"+str(result)+"\n\n") 
            f4.close()

        def save_text_to_file(text):
            #save the actual video text in txt file
            f4=open('videoText_EN.txt','a',encoding='utf-8')
            f4.writelines("The video text : \n"+text+"\n\n") 
            f4.close()
            print ("Done")



    elif lang_of_video=="ar":

        # transcribe text from video
        text = convert_video_to_text(file)

        # summarize text
        summary = summarize_text(text)
        # correct spelling in text
        corrected_text = correct_spelling(text)

        # translate text to arabic
        translated_text = translate_text(text)

        # save text to file
        save_text_to_file(text)

        # render results page
        return render_template('index.html', text=text, summary=summary, corrected_text=corrected_text, translated_text=translated_text)



        def convert_video_to_text(file):
            #path by wav
            path= os.path.splitext(file)[0]+'.wav'

            # use Speech Recognition library to convert the audio of the video to text
            clip=mp.VideoFileClip(file)
            clip.audio.write_audiofile(path)
            r=sr.Recognizer()
            audio=sr.AudioFile(path)
            with audio as source:
                audio_file=r.record(source)
            # return the text
            try:
                text=r.recognize_google(audio_file,language='ar-AR')
                return {"يحتوى الفيديو على : " : r.recognize_google(text)}
            except sr.UnknownValueError as u:
                print(u)
            except sr.RequestError as r:
                print(r)

        def summarize_text(text):
            #summarize text
            #sometimes if the text not big it will make empty text
            #text must not be too small
            summaized_text=summarizer.summarize(text, language ="arabic")
            return {"ملخص الفيديو : ": summaized_text}
            #save the summarized video text in txt file
            f4=open('videoText_AR.txt','a',encoding='utf-8')
            f4.writelines("ملخص الفيديو : \n"+summaized_text+"\n\n") 
            f4.close()
            print ("تم")


        def correct_spelling(text):
            corrector = SpellChecker()
            word = text
            if word in corrector:
                return "صحيح"
            else:
                correct_word = corrector.correction(word)
                return {"الكتابة الصحيحة : ": correct_word}
            #save the corrected video text in txt file
            f4=open('videoText_AR.txt','a',encoding='utf-8')
            f4.writelines("النص بعد التصحيح : \n"+correct_word+"\n\n") 
            f4.close()
            print ("تم")


        def translate_text(text):
            translator = Translator()
            result = translator.translate(text, "english", "arabic")
            return {"النص بعد الترجمة : ": result}
            #save the translated video text in txt file
            f4=open('videoText_AR.txt','a',encoding='utf-8')
            f4.writelines("النص بعد الترجمة :\n"+str(result)+"\n\n") 
            f4.close()
            print ("تم")


        def save_text_to_file(text):
            #save the actual video text in txt file
            f2=open('videoText_AR.txt','a',encoding='utf-8')
            f2.writelines("محتوى الفيديو : \n"+text+"\n\n")
            f2.close()
            print("تم")
                
    else:
        print ("please choose language First")


    
if __name__ == '_main_':
    app.run(debug=True)